In [9]:
import sys
import os

__file__ = 'helper_funcs.py'
file_path = os.path.abspath(os.path.join(os.path.dirname( __file__ ), '..'))
sys.path.append(file_path)

import helper_funcs as hf


In [40]:
# raw_input = '''???.### 1,1,3
# .??..??...?##. 1,1,3
# ?#?#?#?#?#?#?#? 1,3,1,6
# ????.#...#... 4,1,1
# ????.######..#####. 1,6,5
# ?###???????? 3,2,1'''

raw_input = hf.read_file('input.txt')

mod_input = hf.modify_raw(raw_input)

mod_input = [i.split(' ') for i in mod_input]
print(mod_input)
input_dict = {}
for n,i in enumerate(mod_input):
    start = i[0]
    values = [int(x) for  x in i[1].split(',')]
    input_dict.setdefault(n,{'start':start,'values':values})

input_dict

[['..?##?.?.??#???#?', '3,7'], ['???.##??##???#??.#', '1,1,7,1,1,1'], ['?#.??.??#.??#?', '1,1,2,2'], ['?????.?#????', '1,2,1,1'], ['??##???.?.???.', '6,1,2'], ['#??#.??.?.????.#??', '1,1,1,1,3,1'], ['??.??????.?#?', '2,1,3,1'], ['??##?.??#?', '4,3'], ['?#?.???#?????#', '2,4,1,1'], ['#?#??.??##?#?#?#????', '3,13'], ['??????##???', '1,4'], ['??#????????#??.', '7,1'], ['?????##???.', '1,2,1'], ['.??..???????##', '1,1,7'], ['??#?#?.#??.?', '1,1,1,1'], ['?.????????', '3,1'], ['??.??.#?#?#?###?', '2,1,9'], ['???.??????', '3,1,1'], ['?#?#?#???.?#?.??#.', '6,2,2,3'], ['?.??.#.???????', '1,1,3,1'], ['??#???..??#?.?#.', '1,3,1,1,2'], ['??#?#???##?#?#??.', '2,6,1,1,1'], ['???????#??.??.', '1,2'], ['##??????????#?.????#', '9,2,1,2'], ['#?#.#.????#???.', '1,1,1,3,1'], ['?#????.????', '3,1'], ['??#??#?????#.???????', '9,1,2,3'], ['#?.?#???#?##???.#.?', '1,10,1,1'], ['?..??.?????#??#?.', '1,7'], ['?.?#????#?.?', '2,4'], ['.?#????.????', '2,1,1,2'], ['??#??.?#???.#?#', '3,1,4,3'], ['??.?.??#????', '2,

{0: {'start': '..?##?.?.??#???#?', 'values': [3, 7]},
 1: {'start': '???.##??##???#??.#', 'values': [1, 1, 7, 1, 1, 1]},
 2: {'start': '?#.??.??#.??#?', 'values': [1, 1, 2, 2]},
 3: {'start': '?????.?#????', 'values': [1, 2, 1, 1]},
 4: {'start': '??##???.?.???.', 'values': [6, 1, 2]},
 5: {'start': '#??#.??.?.????.#??', 'values': [1, 1, 1, 1, 3, 1]},
 6: {'start': '??.??????.?#?', 'values': [2, 1, 3, 1]},
 7: {'start': '??##?.??#?', 'values': [4, 3]},
 8: {'start': '?#?.???#?????#', 'values': [2, 4, 1, 1]},
 9: {'start': '#?#??.??##?#?#?#????', 'values': [3, 13]},
 10: {'start': '??????##???', 'values': [1, 4]},
 11: {'start': '??#????????#??.', 'values': [7, 1]},
 12: {'start': '?????##???.', 'values': [1, 2, 1]},
 13: {'start': '.??..???????##', 'values': [1, 1, 7]},
 14: {'start': '??#?#?.#??.?', 'values': [1, 1, 1, 1]},
 15: {'start': '?.????????', 'values': [3, 1]},
 16: {'start': '??.??.#?#?#?###?', 'values': [2, 1, 9]},
 17: {'start': '???.??????', 'values': [3, 1, 1]},
 18: {'

In [42]:
def split_no_rem(s:str,delimiter:str)->str:
    final = [i for i in s.split(delimiter) if i != '']
    return final

def check_counter(string:str):
    check_count = []
    for i in (split_no_rem(string,'.')):
        check_count.append(len(i))
    return check_count


def find_combos(starting_pattern:str)->list:

    queue = list([starting_pattern])
    next_queue = []
    for i in range(len(starting_pattern)):
        while len(queue)>0:
            if queue[0][i] in ('.','#'):
                next_queue.append(queue[0])
            elif queue[0][i] == '?':
                for j in ('#','.'):
                    tmp = list(queue[0])
                    tmp[i] = j
                    next_queue.append(''.join(tmp))
            queue.pop(0)
        queue = next_queue
        next_queue = []
    return queue



def count_possible(list_of_patterns:list,value:list)->int:
    total = 0
    for i in list_of_patterns:
        if check_counter(i) == value:
            total += 1
    
    return total



pat = input_dict[0]['start']
values = input_dict[0]['values']
tot = 0
progress_len = len(input_dict)
z = 0
for i in input_dict.keys():
    z+=1
    pat = input_dict[i]['start']
    values = input_dict[i]['values']

    x = find_combos(pat)
    print(f'{z/progress_len*100:.1f}%')
    tot += count_possible(x,values)

print(tot)

# while len(stack)>0:
#     i = 0
#     if stack[i] in ('.','#'):
#         pass
#     if stack[i] == '?':
#         test_dict.setdefault(i,)
#         i+=1
#         test_dict.set_default(i,)

0.1%
0.2%
0.3%
0.4%
0.5%
0.6%
0.7%
0.8%
0.9%
1.0%
1.1%
1.2%
1.3%
1.4%
1.5%
1.6%
1.7%
1.8%
1.9%
2.0%
2.1%
2.2%
2.3%
2.4%
2.5%
2.6%
2.7%
2.8%
2.9%
3.0%
3.1%
3.2%
3.3%
3.4%
3.5%
3.6%
3.7%
3.8%
3.9%
4.0%
4.1%
4.2%
4.3%
4.4%
4.5%
4.6%
4.7%
4.8%
4.9%
5.0%
5.1%
5.2%
5.3%
5.4%
5.5%
5.6%
5.7%
5.8%
5.9%
6.0%
6.1%
6.2%
6.3%
6.4%
6.5%
6.6%
6.7%
6.8%
6.9%
7.0%
7.1%
7.2%
7.3%
7.4%
7.5%
7.6%
7.7%
7.8%
7.9%
8.0%
8.1%
8.2%
8.3%
8.4%
8.5%
8.6%
8.7%
8.8%
8.9%
9.0%
9.1%
9.2%
9.3%
9.4%
9.5%
9.6%
9.7%
9.8%
9.9%
10.0%
10.1%
10.2%
10.3%
10.4%
10.5%
10.6%
10.7%
10.8%
10.9%
11.0%
11.1%
11.2%
11.3%
11.4%
11.5%
11.6%
11.7%
11.8%
11.9%
12.0%
12.1%
12.2%
12.3%
12.4%
12.5%
12.6%
12.7%
12.8%
12.9%
13.0%
13.1%
13.2%
13.3%
13.4%
13.5%
13.6%
13.7%
13.8%
13.9%
14.0%
14.1%
14.2%
14.3%
14.4%
14.5%
14.6%
14.7%
14.8%
14.9%
15.0%
15.1%
15.2%
15.3%
15.4%
15.5%
15.6%
15.7%
15.8%
15.9%
16.0%
16.1%
16.2%
16.3%
16.4%
16.5%
16.6%
16.7%
16.8%
16.9%
17.0%
17.1%
17.2%
17.3%
17.4%
17.5%
17.6%
17.7%
17.8%
17.9%
18.0%
18.1%
18.2%
18.3%
1

In [4]:
sorted([3,1,1]) == sorted([1,1,3])

True

In [43]:
input_dict[691]

{'start': '??????????#????????.', 'values': [2, 10, 2]}